In [1]:
import pyspark
import pandas as pd
import numpy as np
import math

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.feature import PCA
import matplotlib.pyplot as plt

from pyspark.sql.functions import col, mean, column
import matplotlib.pyplot as plt
from pyspark.sql.functions import expr
from pyspark.sql.functions import split
from pyspark.sql import Row
#from pyspark.mllib.recommendation import ALS

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
#ss = SparkSession.builder.master("local").config("spark.driver.memory", "16g").appName("PCAExample1").getOrCreate()
#ss = SparkSession.builder.config("spark.driver.memory", "5g").master("local").appName("PCAExample1").getOrCreate()
ss = SparkSession.builder.master("local").appName("PCAExample1").getOrCreate()

In [4]:
ss.sparkContext.setCheckpointDir("/storage/home/sxs6549/work/Project/scratch")

In [5]:
%%time
df_raw = ss.read.csv("wildfiredb.csv", header=True, inferSchema=True)
#df_raw = spark.read.csv("wildfire100.csv" , header = True, inferSchema = True)
column_names = df_raw.columns

#df_raw = df_raw.drop("acq_date")
df_raw = df_raw.dropna()

CPU times: user 37.6 ms, sys: 9.5 ms, total: 47.1 ms
Wall time: 2min 34s


In [26]:
df_raw[df_raw['frp'] > 1000].show()

+----------+----------+------+---------+-------------+-----------+-----------+--------------+-----------+------------+-------------+------------+-----------+-----------+--------------+-----------+------------+-------------+------------+-----------+-----------+--------------+-----------+------------+-------------+------------+-----------+-----------+--------------+-----------+------------+-------------+------------+-----------+-----------+--------------+-----------+------------+-------------+------------+-----------+-----------+--------------+-----------+------------+-------------+------------+----------+----------+-------------+----------+-----------+------------+-----------+----------+----------+-------------+----------+-----------+------------+-----------+----------+----------+-------------+----------+-----------+------------+-----------+----------+----------+-------------+----------+-----------+------------+-----------+----------+----------+-------------+----------+-----------+-----

In [4]:
#col_list = list(df_raw.columns)
#col_list

In [5]:
#col_list_new = list(set(col_list) - set(['_c0', 'Polygon_ID', 'acq_date', 'frp']))
#col_list_new

In [6]:
#feature_columns = df_raw.columns
col_list = list(df_raw.columns)
feature_columns = list(set(col_list) - set(['_c0', 'Polygon_ID', 'acq_date']))

assembler = VectorAssembler(inputCols = feature_columns, outputCol =  "features")
assembled_data = assembler.transform(df_raw)
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(assembled_data)

scaled_data = scaler_model.transform(assembled_data)

In [ ]:
# Let us perform a hyperparameter sweep
# using 'features'
#k_values = range(5, 15)  # Example: Try K from 2 to 10


#for k_val in k_values:
    #?
    #k = 2

#    pca = PCA(k=k_val, inputCol = "scaled_features", outputCol ="pcaFeatures")
#    model = pca.fit(scaled_data)
#    result = model.transform(scaled_data)

#    explained_variance = model.explainedVariance
#    print("Explained Variance: ", sum(explained_variance), "K: Value: ", k_val)

In [6]:
# Let us perform a hyperparameter sweep
# using 'features'
def variance_search():
    k_values = range(13, 20)  # Example: Try K from 2 to 10


    for k_val in k_values:
        #?
        #k = 2

        pca = PCA(k=k_val, inputCol = "scaled_features", outputCol ="pcaFeatures")
        model = pca.fit(scaled_data)
        result = model.transform(scaled_data)

        explained_variance = model.explainedVariance
        print("Explained Variance: ", sum(explained_variance), "K: Value: ", k_val)
#variance_search()

In [7]:
%%time
pca = PCA(k=36, inputCol="scaled_features", outputCol="pcaFeatures")
model = pca.fit(scaled_data)
result = model.transform(scaled_data)

CPU times: user 65.4 ms, sys: 23.9 ms, total: 89.3 ms
Wall time: 9min 38s


In [10]:
#from pyspark import StorageLevel

#result.persist(StorageLevel.MEMORY_ONLY_SER)

In [ ]:
#explained_variance = model.explainedVariance
#print ("Explained Variance:", sum(explained_variance))


In [ ]:
# Extract PCA loadings
# Assuming pca_model is the name of your PCA model instance
#loadings = model.pc.toArray()

# For simplicity, let's analyze the loadings for the first principal component
#pc1_loadings = loadings[:, 0]

# Pair the loadings with feature names and sort
#feature_importance = sorted(list(zip(feature_columns, pc1_loadings)), key=lambda x: x[1], reverse=True)

#for feature, loading in feature_importance:
#    print(f"{feature}: {loading}")


In [ ]:
#num_cluster_centers = 10

#kmean = KMeans(featuresCol= "scaled_features").setK(num_cluster_centers).setSeed(1)

#model= kmean.fit(scaled_data)

#cluster_centers = model.clusterCenters()
#clustered_data = model.transform(scaled_data)

#evaluator = ClusteringEvaluator()
#silhouetter_score = evaluator.evaluate(clustered_data)

#wcss = model.summary.trainingCost

#cluster_sizes = clustered_data.groupBy("prediction").count()
#cluster_sizes.show()


In [ ]:
#cluster_profiles = clustered_data.groupBy('prediction').mean().collect()

#for row in cluster_profiles:
#    print(f"Cluster {row['prediction']}:")
#    for feature in feature_columns:
#        print(f"{feature}: {row['avg(' + feature + ')']}")
#    print("\n")

In [8]:
def fit_kmeans(df_input,column_name='pcaFeatures',num_cluster_centers=3):
    '''
    Requires
    df_input: spark dataframe with column name "features" having vector of real-valued inputs
    num_cluster_centers: integer that tells the algorithm the value of k
    column_name: column name (string) that indicates which vector of features to choose
    Returns
    cluster_data: spark dataframe with predictions
    silhouette_score: float with silhouette score
    wcss: float within cluster sum of squares
    '''
    # Create a K-Means instance
    kmeans = KMeans(featuresCol=column_name).setK(num_cluster_centers).setSeed(1)
    
    # Fit the K-Means model to the data
    model = kmeans.fit(df_input)
       
    # Get the cluster assignments for each data point
    clustered_data = model.transform(df_input)

    # Evaluate the clustering using the ClusteringEvaluator
    evaluator = ClusteringEvaluator()
    silhouette_score = evaluator.evaluate(clustered_data)

    # Print the WCSS (Within-Cluster Sum of Squares)
    print(f"Silhouette Score: {silhouette_score}")
    wcss = model.summary.trainingCost
    print(f"Within cluster sum of squares: {wcss}")
    
    # Get the cluster sizes and centers
    cluster_sizes = clustered_data.groupBy("prediction").count()
    cluster_sizes.show()
    
    # Return the clustered data
    return clustered_data, silhouette_score, wcss

In [9]:
# Define a function to perform visualization
def visualize_clusters_2D(clustered_data, feature_1, feature_2,  num_cluster_centers=3):
    '''
    Requires:
    clustered_data: dataframe returned from Kmeans fit
    num_cluster_centers: integer number of clusters
    feature_1: string with identifiers for first column name
    feature_2: string with identifiers for second column name
    '''
    
    # Convert the DataFrame to a Pandas DataFrame for visualization
    pandas_df = clustered_data.select(feature_1, feature_2, "prediction").toPandas()
    
    # Extract the cluster assignments
    cluster_assignments = pandas_df["prediction"]
    
    # Extract the indices for the cluster center
    p1 = col_list.index(feature_1)
    p2 = col_list.index(feature_2)
    
    # Create a scatter plot for each cluster
    for cluster_id in range(num_cluster_centers):
        # Generate a random color
        random_color = generate_random_color()        
        cluster_data = pandas_df[pandas_df["prediction"] == cluster_id]
        plt.scatter(cluster_data[feature_1], cluster_data[feature_2], color=[random_color], label=f'Cluster {cluster_id}')
        plt.scatter(cluster_data[feature_1].mean(),cluster_data[feature_2].mean(), color=[random_color], marker='x', s=500)
    
    # Set labels and title
    plt.xlabel(feature_1)
    plt.ylabel(feature_2)
    plt.title("K-Means Clustering")
    #Save figure to folder location
    
    mypath = "/storage/home/sxs6549/work/Project/graphs/"
    if not os.path.isdir(mypath):
       os.makedirs(mypath)
    #fig_name = "figure_frp_" + feature_1
    path = "/storage/home/sxs6549/work/Project/graphs/figure_frp_" + feature_1
    plt.savefig(path)
    
    # Show the legend
    plt.legend()
    
    # Display the plot
    plt.show()

In [10]:
import random
def generate_random_color():
    r = random.random()
    g = random.random()
    b = random.random()
    return (r, g, b)

In [11]:
#def kmeans_loop():

#    k_values = range(3, 13)  # Example: Try K from 2 to 10

#    pca_silhouette_scores = []
#    pca_wcss_scores = []

#    for k in k_values:
#        clustered_data, sil, wcss = fit_kmeans(result,column_name='pcaFeatures',num_cluster_centers=k)
#        pca_silhouette_scores.append(sil)
#        pca_wcss_scores.append(wcss)
#    return pca_silhouette_scores, pca_wcss_scores
#kmeans_loop()

In [ ]:
def kmeans_loop():
    #result.persist()
    k_values = range(3, 13)  # Example: Try K from 2 to 10

    pca_silhouette_scores = []
    pca_wcss_scores = []

    for k in k_values:
        clustered_data, sil, wcss = fit_kmeans(result,column_name='pcaFeatures',num_cluster_centers=k)
        pca_silhouette_scores.append(sil)
        pca_wcss_scores.append(wcss)
    return pca_silhouette_scores, pca_wcss_scores

store_km_vals = kmeans_loop()

Silhouette Score: 0.008574648114709689
Within cluster sum of squares: 569197007.0492767
+----------+-------+
|prediction|  count|
+----------+-------+
|         1| 344191|
|         2| 879620|
|         0|1143369|
+----------+-------+

Silhouette Score: -0.05728107164419523
Within cluster sum of squares: 526429720.02557296
+----------+------+
|prediction| count|
+----------+------+
|         1|504126|
|         3|742278|
|         2|670073|
|         0|450703|
+----------+------+

Silhouette Score: -0.17246700589478528
Within cluster sum of squares: 477797476.61750865
+----------+------+
|prediction| count|
+----------+------+
|         1|226089|
|         3|634163|
|         4|226508|
|         2|590925|
|         0|689495|
+----------+------+

Silhouette Score: -0.18533160969433968
Within cluster sum of squares: 442993153.8509926
+----------+------+
|prediction| count|
+----------+------+
|         1|505354|
|         3|414910|
|         5|212610|
|         4|282795|
|         2|2080

In [ ]:
wcss_scores = store_km_vals[1]

In [ ]:
# Plot the Elbow Method curve
k_values = range(3, 13)
plt.plot(k_values, wcss_scores, marker='o')
plt.title("Elbow Method for Optimal K")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("WCSS Scores")
plt.savefig('elbow_chart')
plt.show()

In [7]:
# Let us perform a hyperparameter sweep
# using 'features'
def kmeans_vis_loop():

    #k_values = range(3, 13)  # Example: Try K from 2 to 10
    k_value = 6
    pca_silhouette_scores = []
    pca_wcss_scores = []

    #for k in k_values:
    clustered_data, sil, wcss = fit_kmeans(result,column_name='pcaFeatures',num_cluster_centers=k_value)
    

    for feature in feature_columns:
        #?
        #k = 2
        # Convert the DataFrame to a Pandas DataFrame for visualization
        feature_1 = feature
        feature_2 = "frp"
        #print("X")
        visualize_clusters_2D(clustered_data, feature_1, feature_2, k_value)
#kmeans_vis_loop()

In [12]:
# First, apply KMeans clustering on the PCA results
#num_cluster_centers = 3
#kmean = KMeans(featuresCol="pcaFeatures", k=num_cluster_centers).setSeed(1)
#kmeans_model = kmean.fit(result)
#clustered_data = kmeans_model.transform(result)

# Extract PCA values and predictions
#pca_values = clustered_data.select("pcaFeatures", "prediction").collect()
#x = [row['pcaFeatures'][0] for row in pca_values]
#y = [row['pcaFeatures'][1] for row in pca_values]
#colors = [row['prediction'] for row in pca_values]

# Plotting
#plt.scatter(x, y, c=colors, cmap='tab10')
#plt.xlabel('Principal Component 1')
#plt.ylabel('Principal Component 2')
#plt.colorbar()
#plt.show()



In [ ]:
# Extract PCA values and predictions
# 2. Run KMeans clustering on the PCA-transformed data
#kmean = KMeans(featuresCol="pcaFeatures", k=num_cluster_centers).setSeed(1)
#kmeans_model = kmean.fit(result)
#clustered_data = kmeans_model.transform(result)

# Confirming that the 'prediction' column exists
#clustered_data.show()

# 3. Visualize the clusters using the first two principal components
#pca_values = clustered_data.select("pcaFeatures", "prediction").collect()
#x = [row['pcaFeatures'][0] for row in pca_values]
#y = [row['pcaFeatures'][1] for row in pca_values]
#colors = [row['prediction'] for row in pca_values]

#plt.scatter(x, y, c=colors, cmap='tab10')
#plt.xlabel('Principal Component 1')
#plt.ylabel('Principal Component 2')
#plt.colorbar()
#plt.show()


In [ ]:
##Random Forest

In [ ]:
#Trial

#from pyspark.ml.feature import VectorAssembler

#feature_list = []
#for col in df.columns:
#    if col == 'label':
#        continue
#    else:
#        feature_list.append(col)

#assembler = VectorAssembler(inputCols=feature_list, outputCol="features")

In [ ]:
#Decision Tree Regression

In [7]:
#%%time
#small dataset, 200MB, to use for decision tree hyper parameter tuning
#df_raw_trial = ss.read.csv("fire_small.csv", header=True, inferSchema=True)

#df_raw = df_raw.drop("acq_date")
#df_raw_trial = df_raw_trial.dropna()

#feature_columns = df_raw.columns

CPU times: user 24.2 ms, sys: 3.93 ms, total: 28.2 ms
Wall time: 6.93 s


In [25]:
#feature_columns = df_raw.columns
#col_list = list(df_raw.columns)
col_list = list(df_raw_trial.columns)
feature_inputs = list(set(col_list) - set(['_c0', 'Polygon_ID', 'acq_date', 'frp']))

assembler_tree = VectorAssembler(inputCols = feature_inputs, outputCol =  "features")
assembled_data_tree = assembler_tree.transform(df_raw_trial)
#scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
#scaler_model = scaler.fit(assembled_data)

#scaled_data = scaler_model.transform(assembled_data)

In [10]:
pca_tree = PCA(k=36, inputCol="features", outputCol="pcaFeatures")
model_tree = pca_tree.fit(assembled_data_tree)
result_tree = model_tree.transform(assembled_data_tree)

In [39]:
"""from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils

from decision_tree_plot.decision_tree_parser import decision_tree_parse
from decision_tree_plot.decision_tree_plot import plot_trees
# Split the data into training and test sets (20% held out for testing)
(trainingData, testData) = result_tree.randomSplit([0.8, 0.2])

# Train a DecisionTree model.
dt = DecisionTreeRegressor(labelCol="frp", featuresCol="pcaFeatures", maxDepth=7, minInstancesPerNode=4)

# Chain indexer and tree in a Pipeline
#pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = dt.fit(trainingData)



In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils

from decision_tree_plot.decision_tree_parser import decision_tree_parse
from decision_tree_plot.decision_tree_plot import plot_trees
# Split the data into training and test sets (20% held out for testing)
(trainingData, testingData) = result_tree.randomSplit([0.8, 0.2])

#Code cell for Part 7
## Initialize a Pandas DataFrame to store evaluation results of all combination of hyper-parameter settings
hyperparams_eval_df = pd.DataFrame( columns = ['max_depth', 'minInstancesPerNode', 'training_rmse', 'testing_rmse',  'Best Model'] )
# initialize index to the hyperparam_eval_df to 0
index =0 
# initialize lowest_error
lowest_testing_rmse = 100000
# Set up the possible hyperparameter values to be evaluated
max_depth_list = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
minInstancesPerNode_list = [2, 3, 4, 5, 6, 7, 8, 9, 20]
#max_depth_list = [2]
#minInstancesPerNode_list = [9]
#labelIndexer = StringIndexer(inputCol="class", outputCol="indexedLabel").fit(data2)
#feature_inputs = list(set(col_list) - set(['_c0', 'Polygon_ID', 'acq_date', 'frp']))
#assembler = VectorAssembler( inputCols=feature_inputs, outputCol="features")
#labelConverter = IndexToString(inputCol = "prediction", outputCol="predictedClass", labels=labelIndexer.labels)
model_path="/storage/home/sxs6549/work/Project/DTmodel_vis"
trainingData.persist()
testingData.persist()

DataFrame[_c0: int, Polygon_ID: int, acq_date: date, frp: double, Neighbour: int, Neighbour_frp: double, CBC2014_max: double, CBC2014_min: double, CBC2014_median: double, CBC2014_sum: double, CBC2014_mode: double, CBC2014_count: int, CBC2014_mean: double, CBD2012_max: double, CBD2012_min: double, CBD2012_median: double, CBD2012_sum: double, CBD2012_mode: double, CBD2012_count: int, CBD2012_mean: double, CBD2016_max: double, CBD2016_min: double, CBD2016_median: double, CBD2016_sum: double, CBD2016_mode: double, CBD2016_count: int, CBD2016_mean: double, CBH2012_max: double, CBH2012_min: double, CBH2012_median: double, CBH2012_sum: double, CBH2012_mode: double, CBH2012_count: int, CBH2012_mean: double, CBH2014_max: double, CBH2014_min: double, CBH2014_median: double, CBH2014_sum: double, CBH2014_mode: double, CBH2014_count: int, CBH2014_mean: double, CBH2016_max: double, CBH2016_min: double, CBH2016_median: double, CBH2016_sum: double, CBH2016_mode: double, CBH2016_count: int, CBH2016_mea

In [ ]:
#code to run after obtaining best params from smaller dataset

best_max_depth = 8
best_minInsPN = 4
dt = DecisionTreeRegressor(labelCol="frp", featuresCol="pcaFeatures", maxDepth=best_max_depth, minInstancesPerNode=best_minInsPN)
#pipeline = Pipeline(stages=[labelIndexer, assembler, dt, predictionConverter])
model = dt.fit(trainingData)
training_predictions = model.transform(trainingData)
testing_predictions = model.transform(testingData)
#evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="f1")
evaluator = RegressionEvaluator(labelCol="frp", predictionCol="prediction", metricName="rmse")
training_rmse = evaluator.evaluate(training_predictions)
testing_rmse = evaluator.evaluate(testing_predictions)
        
print('Using the max_depth =', best_max_depth, ', and minInstancesPerNode = ', \
      best_minInsPN, ', the testing rmse = ', lowest_testing_rmse) 

In [15]:
%%time
for max_depth in max_depth_list:
    for minInsPN in minInstancesPerNode_list:
        seed = 37
        # Construct a DT model using a set of hyper-parameter values and training data
        #dt= DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="features", maxDepth= max_depth, minInstancesPerNode= minInsPN)
        dt = DecisionTreeRegressor(labelCol="frp", featuresCol="pcaFeatures", maxDepth=max_depth, minInstancesPerNode=minInsPN)
        #pipeline = Pipeline(stages=[labelIndexer, assembler, dt, predictionConverter])
        model = dt.fit(trainingData)
        training_predictions = model.transform(trainingData)
        testing_predictions = model.transform(testingData)
        #evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="f1")
        evaluator = RegressionEvaluator(labelCol="frp", predictionCol="prediction", metricName="r2")
        training_rmse = evaluator.evaluate(training_predictions)
        testing_rmse = evaluator.evaluate(testing_predictions)
        # We use 0 as default value of the 'Best Model' column in the Pandas DataFrame.
        # The best model will have a value 1000
        hyperparams_eval_df.loc[index] = [ max_depth, minInsPN, training_rmse, testing_rmse, 0]  
        index = index +1
        if testing_rmse < lowest_testing_rmse :
            best_max_depth = max_depth
            best_minInsPN = minInsPN
            best_index = index -1
            best_parameters_training_rmse = training_rmse
            best_DTmodel= model
            best_tree = decision_tree_parse(best_DTmodel, ss, model_path)
            column = dict( [ (str(idx), i) for idx, i in enumerate(feature_inputs) ])           
            lowest_testing_rmse = testing_rmse
print('The best max_depth is ', best_max_depth, ', best minInstancesPerNode = ', \
      best_minInsPN, ', testing rmse = ', lowest_testing_rmse) 
column = dict([(str(idx), i) for idx, i in enumerate(feature_inputs)])

The best max_depth is  7 , best minInstancesPerNode =  2 , testing rmse =  46.639026827421304
CPU times: user 2.06 s, sys: 385 ms, total: 2.45 s
Wall time: 5min 51s


In [29]:
%%time
for max_depth in max_depth_list:
    for minInsPN in minInstancesPerNode_list:
        seed = 37
        # Construct a DT model using a set of hyper-parameter values and training data
        #dt= DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="features", maxDepth= max_depth, minInstancesPerNode= minInsPN)
        dt = DecisionTreeRegressor(labelCol="frp", featuresCol="pcaFeatures", maxDepth=max_depth, minInstancesPerNode=minInsPN)
        #pipeline = Pipeline(stages=[labelIndexer, assembler, dt, predictionConverter])
        model = dt.fit(trainingData)
        training_predictions = model.transform(trainingData)
        testing_predictions = model.transform(testingData)
        #evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="f1")
        evaluator = RegressionEvaluator(labelCol="frp", predictionCol="prediction", metricName="r2")
        training_rmse = evaluator.evaluate(training_predictions)
        testing_rmse = evaluator.evaluate(testing_predictions)
        # We use 0 as default value of the 'Best Model' column in the Pandas DataFrame.
        # The best model will have a value 1000
        hyperparams_eval_df.loc[index] = [ max_depth, minInsPN, training_rmse, testing_rmse, 0]  
        index = index +1
        if testing_rmse < lowest_testing_rmse :
            best_max_depth = max_depth
            best_minInsPN = minInsPN
            best_index = index -1
            best_parameters_training_rmse = training_rmse
            best_DTmodel= model
            best_tree = decision_tree_parse(best_DTmodel, ss, model_path)
            column = dict( [ (str(idx), i) for idx, i in enumerate(feature_inputs) ])           
            lowest_testing_rmse = testing_rmse
print('The best max_depth is ', best_max_depth, ', best minInstancesPerNode = ', \
      best_minInsPN, ', testing rmse = ', lowest_testing_rmse) 
column = dict([(str(idx), i) for idx, i in enumerate(feature_inputs)])

The best max_depth is  11 , best minInstancesPerNode =  2 , testing rmse =  -0.29173508735513076
CPU times: user 2.16 s, sys: 458 ms, total: 2.62 s
Wall time: 5min 39s


In [28]:
#training_predictions.show(100)

In [20]:
#Code cell for Part 7
best_model_path_part7="/storage/home/sxs6549/work/Project/DT_HPT_cluster_best_part7"

In [21]:
#Code cell for Part 7
best_tree=decision_tree_parse(best_DTmodel, ss, best_model_path_part7)
column = dict([(str(idx), i) for idx, i in enumerate(feature_inputs)])
plot_trees(best_tree, column = column, output_path = '/storage/home/sxs6549/work/Project/DT_HPT_cluster_best_part7.html')

In [22]:
#Code cell for Part 7
# Store the Testing RMS in the DataFrame
hyperparams_eval_df.loc[best_index]=[best_max_depth, best_minInsPN, best_parameters_training_rmse, lowest_testing_rmse, 1000]

In [23]:
output_path_part7 = "/storage/home/sxs6549/work/Project/Lab8_HPT_cluster_part7.csv"
hyperparams_eval_df.to_csv(output_path_part7)  